# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    # 创建dataset
    dataset = data.TensorDataset(*data_arrays)
    # 返回dataloader
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.3435, -0.6573],
         [-2.0744, -0.7294],
         [ 0.1161,  0.1560],
         [-0.0672, -1.4114],
         [-1.9267,  0.2454],
         [ 0.2276, -1.6785],
         [-1.5299, -1.0378],
         [-0.7201,  0.1732],
         [-0.0252,  1.3207],
         [ 0.8270,  0.6098]]),
 tensor([[ 5.7377],
         [ 2.5442],
         [ 3.8839],
         [ 8.8757],
         [-0.4795],
         [10.3540],
         [ 4.6887],
         [ 2.1545],
         [-0.3436],
         [ 3.7801]])]

使用框架的预定义好的层

在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [7]:
net[0].weight.data.normal_(0, 0.01) # 设置权重,即w
net[0].bias.data.fill_(0)   # 设置偏置项，即b

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [8]:
loss = nn.MSELoss()

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。 我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。 当我们需要更复杂的模型时，高级API的优势将大大增加。 当我们有了所有的基本组件，训练过程代码与我们从零开始实现时所做的非常相似。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:

通过调用net(X)生成预测并计算损失l（前向传播）。

通过进行反向传播来计算梯度。

通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [13]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() # 梯度清零，用于防止梯度累积
        l.backward()    # 计算梯度
        trainer.step()  # 更新参数
    with torch.no_grad():
        l = loss(net(features), labels)
        print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000095
epoch 2, loss 0.000096
epoch 3, loss 0.000095


下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [15]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0005, -0.0002])
b的估计误差： tensor([-0.0003])


## 练习
如果我们用nn.MSELoss(reduction=‘sum’)替换 ，nn.MSELoss（）”为了使代码的行为相同，需要怎么更改学习速率？为什么？

答：应该把学习率除以batch_size，因为默认参数是’mean’，换成’sum’需要除以批量数，一般会采用默认，因为这样学习率可以跟batch_size解耦